# FOR THE BOARD

In [ ]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# setting gain #

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray([17,DEFAULT_SPI_PARAMS['output_amp_gain']])
# data = bytearray(100)
cs = machine.Pin(5, machine.Pin.OUT)

# have to turn GPIO 5 off before writing the gain

cs.off() 
spi.write(data)
cs.on() 

# will change lol
ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

# set for frequency the adc should be read and set the size of the array to send every t seconds 
pot_size = 256

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) >= pot_size:
        # fifo
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=pot_size, callback=sample_callback)

requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 7 HZ ###"})
time.sleep(10)
    
#send 7hz data
for i in range(4):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"7":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/7hz", toSend)
    
requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 10 HZ ###"})
time.sleep(10)

#send 10hz data
for i in range(4):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"10":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/10hz", toSend)

requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 12 HZ ###"})
time.sleep(5)

#send 12hz data
for i in range(4):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"12":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/12hz", toSend)

requests.GETRequest("http://192.168.0.37:5001/isCalibrated")

while True:
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/decode", toSend)

# FOR THE PROCESSING

In [ ]:
ds = 1 #downsample averaging size
if ds > 1:
    downsample=True
else:
    downsample=False
freqs = [7,10,12]
fs_synth = 256
fs = int(fs_synth/ds)
over_n_seconds = 1
number_of_samples = fs*over_n_seconds
number_of_train = 4
removeDC = True
applyFilter = True
synth_power = 0
synth_noise = 0

data_file_all = load_array_data_float("/Users/rishil/Desktop/FYP/EEG-decoding/eeg_lib/log/livedata/on_board_downsample_filter1/decoding_data.txt")
# data_file_all = load_array_data_float(r"C:\Users\RISHI\Desktop\FYP\EEG-decoding\eeg_lib\log\livedata\on_board_downsample_filter\decoding_data.txt")
test_7 = data_file_all[:256*10]
test_10 = data_file_all[256*10:256*20]
test_12 = data_file_all[256*20:256*30]

print(len(test_7),len(test_10),len(test_12))

train_7 = load_array_data_float("/Users/rishil/Desktop/FYP/EEG-decoding/eeg_lib/log/livedata/on_board_downsample_filter1/7hz_calibration.txt")
# train_7 = load_array_data_float(r"C:\Users\RISHI\Desktop\FYP\EEG-decoding\eeg_lib\log\livedata\on_board_downsample_filter\7hz_calibration.txt")
train_10 = load_array_data_float("/Users/rishil/Desktop/FYP/EEG-decoding/eeg_lib/log/livedata/on_board_downsample_filter1/10hz_calibration.txt")
# train_10 = load_array_data_float(r"C:\Users\RISHI\Desktop\FYP\EEG-decoding\eeg_lib\log\livedata\on_board_downsample_filter\10hz_calibration.txt")
train_12 = load_array_data_float("/Users/rishil/Desktop/FYP/EEG-decoding/eeg_lib/log/livedata/on_board_downsample_filter1/12hz_calibration.txt")
# train_12 = load_array_data_float(r"C:\Users\RISHI\Desktop\FYP\EEG-decoding\eeg_lib\log\livedata\on_board_downsample_filter\12hz_calibration.txt")

print(len(train_7),len(train_10),len(train_12))

data_file_7 = flatten([train_7[:fs_synth*number_of_train], test_7])
data_file_10 = flatten([train_10[:fs_synth*number_of_train], test_10])
data_file_12 = flatten([train_12[:fs_synth*number_of_train], test_12])

values_7 = prepare_data(data_file_7, 7, fs, fs_synth, number_of_samples, number_of_train, remove_DC=removeDC, apply_filter=applyFilter, downsample=downsample, ds_avg=ds, synth_power=synth_power, noise_power=synth_noise)
values_10 = prepare_data(data_file_10, 10, fs, fs_synth, number_of_samples, number_of_train, remove_DC=removeDC, apply_filter=applyFilter, downsample=downsample, ds_avg=ds, synth_power=synth_power, noise_power=synth_noise)
values_12 = prepare_data(data_file_12, 12, fs, fs_synth, number_of_samples, number_of_train, remove_DC=removeDC, apply_filter=applyFilter, downsample=downsample, ds_avg=ds, synth_power=synth_power, noise_power=synth_noise)

print(values_12.shape, values_10.shape, values_7.shape)

data_packed = np.array([values_7, values_10, values_12])

print(data_packed.shape)

train_test_idxs = generate_train_test_idxs(data_packed, number_of_train)
print(len(train_test_idxs))
print((train_test_idxs[0]))
test_gcca_mset(data_packed, train_test_idxs, freqs, fs, number_of_samples, number_runs=1)